In [ ]:
## Running this file locally might be the best option 


## Libraries

In [ ]:
import nltk
import os
import pickle
import re
import shlex
# import stanza
import subprocess
import time

import pandas as pd
import numpy as np

from nltk import word_tokenize
from nltk.tokenize import SyllableTokenizer
from subprocess import Popen, PIPE
from tqdm import tqdm

## n-grams gerator function


In [ ]:
# This function generates n-grams generated from a string.
#
# param s: is the string passed into this function
# param n: is the n in n-grams
# returns: the n-grams

def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s) # \s white, ^ : negation
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

## Data

In [ ]:
res = pd.read_excel('final_results_data_.xlsx')
res.shape

(10292, 12)

In [ ]:
results = res['full'].tolist()
results[0]

'Incoming Webhooks — Mattermost 5.11 documentationUse incoming webhooks to post messages to Mattermost public channels, private  .... Couldn\'t find the channel : Indicates that the channel doesn\'t exist or is invalid.  ... You have space around JSON separator colons, payload={\\Hello\\" : \\"test\\"} or\xa0..."'

## n-grams

In [ ]:
posData = [] # -- stores list of pos of each word eg: [array(['NN', 'NN', 'NNS'], dtype='<U8'),
for document in results:
    text = nltk.word_tokenize(document)
    tags = np.array(nltk.pos_tag(text)).flatten()
    posData.append(tags[1::2]) # -- select a pos 
#     print(document)
#     print(text)
#     print(tags)
#     print(posData)


posMod = [] # -- well presented pos of each word  eg: 'NN NN NNS '. --> the pos of each result is an element of the list

for pos in posData: 
    string = []
    for entry in pos:
        string += str(entry) + " "
#         print(string)
    posMod.append("".join(string))

    
posUni = []
posBi = []
posTri = []

for document in posMod:
    doc = generate_ngrams(document,1)
    posUni.append(doc)

for document in posMod:
    doc = generate_ngrams(document,2)
    posBi.append(doc)

for document in posMod:
    doc = generate_ngrams(document,3)
    posTri.append(doc)  
    
posDF = pd.DataFrame(results)

posDF['all'] = posMod
posDF['uniPos'] = posUni
posDF['biPos'] = posBi
posDF['triPos']= posTri
posDF = posDF.rename(columns={0: "results"})

allSessionsuni = pd.concat([posDF,pd.get_dummies(posDF['uniPos'].apply(pd.Series).stack()).sum(level=0)],axis=1).drop(['uniPos', 'all', 'biPos', 'triPos'],axis=1)
allSessionsbi = pd.concat([posDF,pd.get_dummies(posDF['biPos'].apply(pd.Series).stack()).sum(level=0)],axis=1).drop(['biPos', 'uniPos', 'all', 'triPos'],axis=1)
allSessionstri = pd.concat([posDF,pd.get_dummies(posDF['triPos'].apply(pd.Series).stack()).sum(level=0)],axis=1).drop(['uniPos', 'all', 'biPos', 'triPos'],axis=1)

<ipython-input-16-65071a15c156>:46: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  allSessionsuni = pd.concat([posDF,pd.get_dummies(posDF['uniPos'].apply(pd.Series).stack()).sum(level=0)],axis=1).drop(['uniPos', 'all', 'biPos', 'triPos'],axis=1)
<ipython-input-16-65071a15c156>:47: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  allSessionsbi = pd.concat([posDF,pd.get_dummies(posDF['biPos'].apply(pd.Series).stack()).sum(level=0)],axis=1).drop(['biPos', 'uniPos', 'all', 'triPos'],axis=1)
<ipython-input-16-65071a15c156>:48: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. 

In [ ]:
allSessionsuni

,0,cc,cd,dt,ex,fw,in,jj,jjr,jjs,...,uh,vb,vbd,vbg,vbn,vbp,vbz,wdt,wp,wrb
0,Incoming Webhooks — Mattermost 5.11 documentat...,2,1,2,0,0,2,4,0,0,...,0,3,0,2,0,1,3,0,0,0
1,“Hello” Is Facebook's New Android-Only Social ...,1,2,3,0,0,4,3,0,0,...,0,0,0,3,0,0,2,0,0,0
2,Walkthrough | HTTP Event CollectorHEC tokens a...,3,0,3,0,0,7,4,0,0,...,0,2,1,1,1,1,0,0,0,0
3,College Scholarships - Scholarships.comMost co...,3,0,2,0,0,2,2,0,0,...,0,2,0,1,0,2,1,0,0,0
4,How to start sending email — Mailgun API docum...,1,0,1,0,0,2,6,1,0,...,0,3,0,4,0,1,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,"Science, Technology, Engineering, and Math, in...",2,0,1,0,0,2,2,0,0,...,0,1,0,1,0,0,0,1,0,0
10288,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,2,0,2,0,0,2,4,0,0,...,0,0,0,0,1,0,2,1,0,0
10289,Stem Definition & Meaning - Merriam-WebsterKid...,1,2,3,0,0,6,1,0,0,...,0,3,0,0,0,0,2,0,0,0
10290,Driven By StemStem Holdings has positioned its...,0,0,2,0,0,4,0,0,0,...,0,1,1,1,2,0,1,0,0,0


In [ ]:
allSessionsbi

,0,cc cc,cc cd,cc dt,cc ex,cc in,cc jj,cc jjr,cc jjs,cc md,...,wrb pdt,wrb prp,wrb rb,wrb to,wrb vb,wrb vbd,wrb vbg,wrb vbn,wrb vbp,wrb vbz
0,Incoming Webhooks — Mattermost 5.11 documentat...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,“Hello” Is Facebook's New Android-Only Social ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Walkthrough | HTTP Event CollectorHEC tokens a...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,College Scholarships - Scholarships.comMost co...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,How to start sending email — Mailgun API docum...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,"Science, Technology, Engineering, and Math, in...",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10288,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10289,Stem Definition & Meaning - Merriam-WebsterKid...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10290,Driven By StemStem Holdings has positioned its...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
allSessionstri

,0,cc cc nn,cc cc nnp,cc cc vb,cc cd cc,cc cd cd,cc cd dt,cc cd in,cc cd jj,cc cd nn,...,wrb vbp prp,wrb vbp rb,wrb vbp vbg,wrb vbz cd,wrb vbz dt,wrb vbz jj,wrb vbz nn,wrb vbz nnp,wrb vbz prp,wrb vbz vb
0,Incoming Webhooks — Mattermost 5.11 documentat...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,“Hello” Is Facebook's New Android-Only Social ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Walkthrough | HTTP Event CollectorHEC tokens a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,College Scholarships - Scholarships.comMost co...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,How to start sending email — Mailgun API docum...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,"Science, Technology, Engineering, and Math, in...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10288,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10289,Stem Definition & Meaning - Merriam-WebsterKid...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10290,Driven By StemStem Holdings has positioned its...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
